In [1]:
def clip(tmp1):
    import subprocess
    import shlex  # 导入 shlex 模块
    # 使用 shlex.quote 来转义 inp 字符串
    tmp2 = str(tmp1)
    safe_str = shlex.quote(tmp2)
    subprocess.run('echo {} | wclip'.format(safe_str), shell=True)  

def cvin(k):
    clip(In[k])
    
import numpy as np
import matplotlib.pyplot as plt

import time 
#from tqdm import tqdm  # tqdm是显示循环进度条的库
from tqdm.notebook import tqdm #推荐在jupyter中使用自带的进度条
import copy #复制方法

#-------------------------------------------------------------------
np.random.seed(0) #重置种子为0
np.set_printoptions(precision=3, suppress=True, linewidth=100)#格式化输出
#-------------------------------------------------------------------

import rl_utils
import random
import gymnasium as gym
import collections
import torch
import torch.nn.functional as F
import os


## 模仿学习：
很多现实场景中，**奖励函数并未给定：**
假设存在一个专家智能体，其策略可以看成最优策略，就可以直接模仿这个专家，并且不需要用到环境提供的奖励信号。<br>
**模仿学习（imitation learning）** 研究的便是这一类问题，在模仿学习的框架下，专家能够提供一系列状态动作对${(s_t,a_t)}$，通常分为三类：
1. 行为克隆（behavior cloning，BC）
2. 逆强化学习（inverse RL）
3. 生成式对抗模仿学习（generative adversarial imitation learning，GAIL）

主要介绍行为克隆方法和生成式对抗模仿学习方法，逆强化学习由于其计算复杂度较高，实际应用的价值较小，但是也有优点：<br>其核心思想是从专家的演示中推断出一个奖励函数，然后使用这个奖励函数进行强化学习，比较如下：

1. **学习目标**：
   - **BC**：BC的目标是直接模仿专家的行为。它不涉及环境中的奖励信号，而只是试图最小化与专家演示之间的行为差异
   - **RL**：RL的目标是学习一个策略，该策略能够最大化长期的累积奖励。RL通过与环境的交互以及环境给予的奖励信号进行学习

2. **数据来源**：
   - **BC**：BC需要专家的演示数据
   - **RL**：RL通常不需要专家的演示，而是通过智能体与环境的交互来收集数据

3. **泛化能力**：
   - **BC**：BC可能在面对与专家演示不完全匹配的新情境时泛化能力较弱。
   - **RL**：如果正确地进行了训练，RL通常可以更好地泛化到未见过的状态。

4. **安全性和稳定性**：
   - **BC**：由于BC是直接模仿专家的行为，如果专家的演示数据中存在噪音或错误，BC可能会学习到这些不良行为
   - **RL**：RL的学习过程可能会探索不安全或不稳定的行为，但它也可以利用奖励信号来调整和改进

5. **计算复杂性**：
   - **BC**：BC通常涉及监督学习，计算复杂性相对较低
   - **RL**：RL，特别是深度RL，可能需要大量的样本和计算资源

#### 选择BC or RL ?
   - **BC**：当获取专家演示相对容易，而与环境的交互代价较高时，BC可能是一个好的选择
   - **RL**：当环境提供明确的奖励信号，或者专家演示难以获得时，RL是更好的选择
   
---

## A.行为克隆BC
> 不学习任何专家的奖励函数，状态分布——就单纯只要确保在专家的环境中"尽量像专家"即可，遇到其他状态就直接歇菜
1. 策略完全不需要环境交互

**行为克隆（BC）** 就是直接使用监督学习方法，将专家数据中$(s_t,a_t)$的$s_t$视作样本输入，$a_t$视为标签，学习的目标为：
$$
\theta^{*}=\arg \min _{\theta} \mathbb{E}_{(s, a) \sim B}\left[\mathcal{L}\left(\pi_{\theta}(s), a\right)\right]
$$
其中，B是专家数据集，$\mathcal{L}$是对应监督学习框架下的损失函数，一般常用的损失函数为 **负对数似然损失（Negative Log Likelihood, NLL）**：
$$
\mathcal{L}\left(\pi_{\theta}(s), a\right) = -\log \pi_{\theta}(a|s)
$$
通俗而言，就是在面对给定(a,s)时，$\pi_theta(a | s)$应该足够大

#### 用途：高起点预训练
> 由于 BC 的实现十分简单，因此在很多实际场景下它都可以作为策略预训练的方法，BC 能使得策略无须在较差时仍然低效地通过和环境交互来探索较好的动作，而是通过模仿专家智能体的行为数据来快速达到较高水平，为接下来的强化学习创造一个高起点。

#### 明显缺点：复合误差
通过 BC 学习得到的策略在和环境交互过程中 **不可能完全学成最优**，只要存在一点偏差，就有可能导致胡乱选择。最终，该策略在真实环境下不能得到比较好的效果，这被称为行为克隆的 **复合误差（compounding error）问题**


### |训练方法：BC转「分类问题」最大似然估计
训练方法：将专家$(s_t,a_t)$的$a_t$视作标签，BC转为监督学习经典「分类问题」->采用最大似然估计训练


---

##  B.生成式对抗模仿学习GAIL
> 本质上是模仿专家的占用度量$\rho_E(s,a)$，尽量确保在所有(s,a)中占用度量$\rho_\pi(s,a)$接近$\rho_E(s,a)$
1. 策略需要和环境交互

**生成式对抗模仿学习（generative adversarial imitation learning，GAIL）** 是 2016 年由斯坦福大学研究团队提出的基于生成式对抗网络的模仿学习，它诠释了—— **_生成式对抗网络的本质其实就是模仿学习_**
<br>GAIL确实可以看作是行为克隆BC的一个扩展，引入了一个额外的判别器层，使得目标不那么直接了，核心在于：生成器&判别器

### |生成器（generator）
给定一个状态，策略会输出这个状态下应该采取的动作，其实就是策略$\pi$

### |判别器（discriminator）
用来判断状态对(s,a)究竟是来自于专家还是智能体，"判别器D目标是尽量将专家数据的输出靠近 0，将模仿者策略的输出靠近 1，这样就可以将两组数据分辨开来"，于是损失函数为：
$$
\mathcal{L}(\phi)=-\mathbb{E}_{\rho_{\pi}}\left[\log D_{\phi}(s, a)\right]-\mathbb{E}_{\rho_{E}}\left[\log \left(1-D_{\phi}(s, a)\right)\right]
$$
其中$\phi$是判别器D的参数
> **注意：损失函数混合了两个交叉熵，相当于判别器需要同时拟合两个分布**

### "对抗"生成器欺骗判别器：
模仿者策略的目标就是其交互产生的轨迹能被判别器误认为专家轨迹，因此使用D的输出来作为奖励函数来训练模仿者策略，奖励函数为：
$$
r(s, a)=-\log D(s, a)
$$
于是，我们可以用任意强化学习算法，使用这些数据继续训练模仿者策略，最终，模仿者策略生成的数据分布将接近真实的专家数据分布

## 训练步骤:

#### 相应标准名称(方便debug)
PolicyNet ValueNet
actor critic
actor_optimizer
gamma
lmbda
epochs 一条序列的数据用于训练轮数
eps 
device

truncated
td_target td_delta
log_probs old_log_probs
surr1 surr2
actor_loss

actor_lr
lambda
gamma
total_epochs 迭代轮数
total_episodes 一轮训练多少次游戏

eps
epochs
device

env

hideen_dim state_dim action_dim

PPO

CKP_PATH

def sample_expert_data
n_sample random_index expert_s/a 

BehaviorClone
policy
optimizer

def learn

def take_action

def test_agent

lr
bc_agent
n_iterations 
batch_size
test_returns

===============
Discriminator
GAIL
discriminator

### 训练框架
def train_on_policy_agent
env agent s_epoch total_epochs s_episode total_episodes return_list ckp_path
start_time end_time
best_score
if return_list
epoch->s_epoch | total_epoch
transition_dict
episode_return | if ->actor_best_weight|
torch_save

----pbar

### 1.生成专家数据
使用PPO算法训练一个表现良好的专家模型
#### 1.1训练PPO专家
#### 1.2采样专家的序列

### 2.行为克隆BC：

#### 2.1「策略网络」进行模仿

### 3.生成式对抗模仿学习GAIL

#### 3.1首先训练判别器，GAIL类
#### 3.2接着生成器：BC「策略网络」+ PPo「价值网络」

## 总结：
在数据样本有限的情况下，BC 不能学习到最优策略，但是 GAIL 在相同的专家数据下可以取得非常好的结果
- 这一方面归因于 GAIL 的训练目标（拉近策略和专家的占用度量）十分贴合模仿学习任务的目标，避免了 BC 中的复合误差问题
- 另一方面得益于 GAIL 训练中，策略可以和环境交互出更多的数据，以此训练判别器，进而生成对 **基于策略“量身定做”** 的指导奖励信号

## C.逆强化学习IRL
> 其核心思想是从专家的演示中推断出一个 **奖励函数**，然后使用这个奖励函数进行强化学习，通过智能体与环境的交互来收集数据
1. 策略需要和环境交互

IRL假设环境的奖励函数应该使得专家轨迹获得最高的奖励值，进而学习背后的奖励函数，最后基于该奖励函数做正向强化学习，从而得到模仿策略。